In [65]:
import open3d as o3d
import os



raw_dir = 'datasets/Thingi10K/raw_meshes/'
graph_dir = 'datasets/Thingi10K/graphs/'


TARGET_SIZE = 5000


def raw_data_file_path(id: int) -> str:
    return f'{raw_dir}{id}'


def all_raw_files():
    return [f for f in os.listdir(raw_dir) if f.endswith(".stl")]


def get_mesh(file_path):
    return o3d.io.read_triangle_mesh(file_path)

In [49]:
from tqdm import tqdm
import random

def data_loader(mesh_count: int | None = None):
    stl_files = all_raw_files()
    if mesh_count == None:
        mesh_count = len(stl_files)
    else:
        random.shuffle(stl_files)

    meshes = []

    for i in tqdm(range(mesh_count), desc="Processing STL Files"):
        file_path = os.path.join(raw_dir, stl_files[i])
        try:
            meshes.append(get_mesh(file_path))
        except Exception as inst:
            print('Error with ', file_path)
            print(inst)
    
    return meshes

mesh_data = data_loader(500)


Processing STL Files: 100%|██████████| 500/500 [00:28<00:00, 17.32it/s]


In [66]:
import numpy as np


def draw_multiple_meshes(mesh_list):
    for i, (mesh, color) in enumerate(zip(mesh_list, [[1, 0, 0], [0, 1, 0]])):
        translation = np.array([i * max(mesh.get_max_bound() - mesh.get_min_bound()) * 1.2, 0, 0])
        mesh.translate(translation, relative=False)

        mesh.paint_uniform_color(color)
    
    # Draw all meshes in one window
    o3d.visualization.draw_geometries(mesh_list, mesh_show_wireframe=True)

In [94]:
import pymeshlab


def preprocess_mesh(mesh, target_vertices=TARGET_SIZE, simplification_ratio=0.1):
    
    # Normalize mesh to be centered at the origin and fit inside a unit sphere
    mesh = mesh.translate(-mesh.get_center())
    mesh.scale(1 / np.max(mesh.get_max_bound() - mesh.get_min_bound()), center=mesh.get_center())

    # # Calculate the number of vertices to remove
    # num_vertices_to_reduce = len(mesh.triangles) - target_vertices
    # num_simplification_steps = int(num_vertices_to_reduce * simplification_ratio)

    # # Perform simplification in steps
    # for _ in range(num_simplification_steps):
    #     mesh = mesh.simplify_quadric_decimation(target_number_of_triangles=len(mesh.triangles) - num_vertices_to_reduce)
    #     # Check for potential errors or stopping conditions
    #     if len(mesh.triangles) <= target_vertices:
    #         break
    # mesh = mesh.simplify_quadric_decimation(target_number_of_triangles=TARGET_SIZE)

    o3d.io.write_triangle_mesh("path_to_save.obj", mesh)

    numFaces = 100 + 2*TARGET_SIZE

    ms = pymeshlab.MeshSet()
    ms.load_new_mesh("path_to_save.obj")

    # pymeshlab.print_filter_list()

    while (ms.current_mesh().vertex_number() > TARGET_SIZE):
        ms.apply_filter('meshing_decimation_quadric_edge_collapse', targetfacenum=numFaces, preservenormal=True)
        print("Decimated to", numFaces, "faces mesh has", ms.current_mesh().vertex_number(), "vertex")
        #Refine our estimation to slowly converge to TARGET vertex number
        numFaces = numFaces - (ms.current_mesh().vertex_number() - TARGET_SIZE)

    # Assuming 'ms' is your PyMeshLab MeshSet with the mesh loaded
    ms.save_current_mesh("path_to_export.obj")

    # Read the mesh file into Open3D
    mesh = o3d.io.read_triangle_mesh("path_to_export.obj")

    return mesh


meshes = [random.choice(mesh_data)]
meshes.append(preprocess_mesh(meshes[0]))

print('original size = ', len(meshes[0].vertices), len(meshes[0].triangles))
print('new size = ', len(meshes[1].vertices), len(meshes[1].triangles))

draw_multiple_meshes(meshes)


# o3d.visualization.draw_geometries([meshes[0]])
# o3d.visualization.draw_geometries([meshes[1]])




original size =  298 612
new size =  298 612
